# Solving an accessibility glitch with UrbanAccess walk + transit networks

Sam Maurer - October 2020

This notebook investigates a reported issue where adding a subway network had the unexpected effect of _reducing_ accessibility (jobs within 15 minutes) for certain locations. 

My original suspicion was that the issue could be related to the contraction hierarchy heuristic that Pandana uses to speed up shortest-path calculations -- fast links are prioritized in the network, so perhaps adding a subway caused some routings to shift to rail even when it wasn't strictly optimal. But this turns out not to be the problem (see separate notebook demonstrating that). 

Here's what it is: When job counts are linked to the network, they're assigned to whatever node is closest to the coordinates from the job data. Usually this is a road intersection, but sometimes it's a transit station. By design, there's extra impedance between a road and a neighboring transit station, meant to capture the wait time between trains. 

So when the rail network is added, some small number of jobs end up assigned to rail stations. And it takes people longer to reach those jobs (people not arriving by train, at least) than it did previously, because of the impedance. 

The solution, demonstrated below, is to make sure jobs are only associated with normal street nodes, not transit nodes. The context we're looking at is Oakland, California and includes the walking network (from Open Street Map), the bus network (ACTransit), and the rail network (BART).

In [1]:
import numpy as np
import pandas as pd

import pandana
import urbanaccess

In [2]:
print(np.__version__)
print(pd.__version__)
print(pandana.__version__)
print(urbanaccess.__version__)

1.19.1
1.1.2
0.5.1
0.2.1


In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## 1. Load job data

This code is copied from the original issue report.

Download this data file here: https://www.dropbox.com/s/q2h5ww119xrebur/bay_area_demo_data.h5?dl=1

In [4]:
blocks = pd.read_hdf('data/bay_area_demo_data.h5','blocks')
# remove blocks that contain all water
blocks = blocks[blocks['square_meters_land'] != 0]
print('Total number of blocks: {:,}'.format(len(blocks)))
blocks.head(3)

Total number of blocks: 107,080


,x,y,res_rents,res_values,square_meters_land,households,persons,workers,children,cars,income,jobs,54,71,81,51,23,3133,42,52,56,4445,62,61,53,72,92,4849,55,22,11,21
060014001001000,-122.231654,37.879012,2201.1,1100001.1,590336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
060014001001001,-122.234077,37.881846,2201.1,1100001.1,22089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
060014001001002,-122.229372,37.880514,2201.1,1100001.1,9433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Subset for Oakland:

In [5]:
bbox = (-122.355881,37.632226,-122.114775,37.884725)

lng_max, lat_min, lng_min, lat_max = bbox
outside_bbox = blocks.loc[~(((lng_max < blocks["x"]) & (blocks["x"] < lng_min)) & ((lat_min < blocks["y"]) & (blocks["y"] < lat_max)))]
blocks_subset = blocks.drop(outside_bbox.index)
print('Total number of subset blocks: {:,}'.format(len(blocks_subset)))

Total number of subset blocks: 11,824


## 2. Replicate problematic accessibilities

Download the two network files here. UrbanAccess expects them to be inside a 'data' folder.
- https://www.dropbox.com/s/sfhdx1zy6ul9v2j/actransit_with_headways.h5?dl=1
- https://www.dropbox.com/s/baqzsn587xwjxen/actransit_bart_with_headways.h5?dl=1

First, load the walk + ACTransit network:

In [6]:
ua_net = urbanaccess.network.load_network(filename='actransit_with_headways.h5')

Successfully read store: data/actransit_with_headways.h5 with the following keys: ['/edges', '/nodes']
Successfully read store: data/actransit_with_headways.h5 with the following keys: ['/edges', '/nodes']


In [7]:
%%time
net = pandana.Network(ua_net.net_nodes["x"],
                      ua_net.net_nodes["y"],
                      ua_net.net_edges["from_int"],
                      ua_net.net_edges["to_int"],
                      ua_net.net_edges[["weight"]], 
                      twoway=False)

CPU times: user 1min 58s, sys: 2.69 s, total: 2min 1s
Wall time: 16.3 s


### Associate jobs with nodes, and calculate accessibilities

In [8]:
blocks_subset['node_id'] = net.get_node_ids(blocks_subset['x'], blocks_subset['y'])
net.set(blocks_subset.node_id, variable = blocks_subset.jobs, name='jobs')

In [9]:
jobs_15 = net.aggregate(15, type='sum', decay='linear', name='jobs')

In [10]:
ua_nodes1 = ua_net.net_nodes.copy()
ua_nodes1['jobs_15_act'] = jobs_15

In [11]:
ua_nodes1.head(3)

,id,x,y,net_type,jobs_15_act
id_int,,,,,
1,0600390_ac_transit_J-141_ac_transit,-122.292298,37.848981,NaN,3433.030506
2,0601170_ac_transit_J-141_ac_transit,-122.292158,37.846713,NaN,3564.098974
3,0600190_ac_transit_J-141_ac_transit,-122.295368,37.846228,NaN,3158.065181


### Repeat for walk + ACTransit + BART network

In [12]:
ua_net = urbanaccess.network.load_network(filename='actransit_bart_with_headways.h5')

Successfully read store: data/actransit_bart_with_headways.h5 with the following keys: ['/edges', '/nodes']
Successfully read store: data/actransit_bart_with_headways.h5 with the following keys: ['/edges', '/nodes']


In [13]:
%%time
net = pandana.Network(ua_net.net_nodes["x"],
                      ua_net.net_nodes["y"],
                      ua_net.net_edges["from_int"],
                      ua_net.net_edges["to_int"],
                      ua_net.net_edges[["weight"]], 
                      twoway=False)

CPU times: user 2min 3s, sys: 3.25 s, total: 2min 6s
Wall time: 17.2 s


In [14]:
blocks_subset['node_id'] = net.get_node_ids(blocks_subset['x'], blocks_subset['y'])
net.set(blocks_subset.node_id, variable = blocks_subset.jobs, name='jobs')

In [15]:
jobs_15 = net.aggregate(15, type='sum', decay='linear', name='jobs')

In [16]:
ua_nodes2 = ua_net.net_nodes.copy()
ua_nodes2['jobs_15_act_bart'] = jobs_15

In [17]:
ua_nodes2.head(3)

,id,x,y,net_type,jobs_15_act_bart
id_int,,,,,
1,FTVL_bay_area_rapid_transit_05_bay_area_rapid_...,-122.224274,37.774963,NaN,10300.463643
2,COLS_bay_area_rapid_transit_05_bay_area_rapid_...,-122.197273,37.754006,NaN,8608.617062
3,SANL_bay_area_rapid_transit_05_bay_area_rapid_...,-122.161311,37.722619,NaN,6321.116765


### Merge the accessibilities and compare

In [18]:
ua_nodes1_clean = ua_nodes1.loc[ua_nodes1.net_type == 'walk', ['id','x','y','jobs_15_act']]
ua_nodes2_clean = ua_nodes2.loc[ua_nodes2.net_type == 'walk', ['id','jobs_15_act_bart']]

nodes = pd.merge(ua_nodes1_clean, 
                 ua_nodes2_clean, 
                 how = 'inner', 
                 on = 'id')

In [19]:
nodes['jobs_15_ratio'] = nodes.jobs_15_act_bart / nodes.jobs_15_act

In [20]:
nodes.describe()

,x,y,jobs_15_act,jobs_15_act_bart,jobs_15_ratio
count,55440.000000,55440.000000,55440.000000,55440.000000,54577.000000
mean,-122.220596,37.784235,1843.589963,1918.135519,1.039537
std,0.055707,0.060465,3190.372252,3332.920532,0.372502
min,-122.342039,37.631875,0.000000,0.000000,0.488894
25%,-122.267031,37.739285,292.714193,293.846367,1.000000
50%,-122.232524,37.787570,878.969028,888.557053,1.000000
75%,-122.174038,37.832181,1867.369856,1940.829419,1.000000
max,-122.107708,37.891111,27177.999511,30860.928071,11.830064


In [21]:
print(len(nodes))
print(len(nodes.loc[nodes.jobs_15_ratio < 1]))

55440
5769


#### 10% of nodes have lower accessibility after adding BART to the network, which is clearly wrong.

## 3. Repeat, linking jobs ONLY to the non-transit nodes

In [22]:
ua_net = urbanaccess.network.load_network(filename='actransit_with_headways.h5')

Successfully read store: data/actransit_with_headways.h5 with the following keys: ['/edges', '/nodes']
Successfully read store: data/actransit_with_headways.h5 with the following keys: ['/edges', '/nodes']


In [23]:
%%time
net = pandana.Network(ua_net.net_nodes["x"],
                      ua_net.net_nodes["y"],
                      ua_net.net_edges["from_int"],
                      ua_net.net_edges["to_int"],
                      ua_net.net_edges[["weight"]], 
                      twoway=False)

CPU times: user 1min 44s, sys: 13.7 s, total: 1min 58s
Wall time: 15.7 s


### Limit to just the OSM nodes

We link jobs to nodes on the Pandana side, but at that point the metadata about which nodes are from the walking network vs. the transit network is lost. So I'm creating a second Pandana network with just the OSM nodes, in order to associate the job counts with appropriate node ID's.

In [24]:
osm_node_ids = ua_net.net_nodes.loc[ua_net.net_nodes.net_type == 'walk'].index.values
print(len(ua_net.net_nodes))
print(len(osm_node_ids))

59099
55440


In [25]:
osm_nodes = ua_net.net_nodes.reindex(osm_node_ids)  # filters for rows with index in list of values

In [26]:
osm_edges = ua_net.net_edges.loc[ua_net.net_edges.from_int.isin(osm_node_ids) &
                                 ua_net.net_edges.to_int.isin(osm_node_ids)]

In [27]:
%%time
osm_net = pandana.Network(osm_nodes["x"], 
                          osm_nodes["y"],
                          osm_edges["from_int"],
                          osm_edges["to_int"],
                          osm_edges[["weight"]], 
                          twoway=False)

CPU times: user 9.49 s, sys: 1.18 s, total: 10.7 s
Wall time: 1.68 s


Assign node ids to job counts using the smaller network, but then "set" the jobs onto the full network:

In [28]:
blocks_subset['node_id'] = osm_net.get_node_ids(blocks_subset['x'], blocks_subset['y'])

In [29]:
net.set(blocks_subset.node_id, variable = blocks_subset.jobs, name='jobs')

Calculate accessibility on the full network:

In [30]:
jobs_15 = net.aggregate(15, type='sum', decay='linear', name='jobs')

In [31]:
ua_nodes1 = ua_net.net_nodes.copy()
ua_nodes1['jobs_15_act'] = jobs_15

In [32]:
ua_nodes1.head(3)

,id,x,y,net_type,jobs_15_act
id_int,,,,,
1,0600390_ac_transit_J-141_ac_transit,-122.292298,37.848981,NaN,3469.682774
2,0601170_ac_transit_J-141_ac_transit,-122.292158,37.846713,NaN,3626.442572
3,0600190_ac_transit_J-141_ac_transit,-122.295368,37.846228,NaN,3234.868380


### Do the same thing for the walk + ACTransit + BART network

We need to generate the OSM subset of this network separately too, I think, because the nodes will have different ids assigned in UrbanAccess.

In [33]:
ua_net = urbanaccess.network.load_network(filename='actransit_bart_with_headways.h5')

Successfully read store: data/actransit_bart_with_headways.h5 with the following keys: ['/edges', '/nodes']
Successfully read store: data/actransit_bart_with_headways.h5 with the following keys: ['/edges', '/nodes']


In [34]:
%%time
net = pandana.Network(ua_net.net_nodes["x"],
                      ua_net.net_nodes["y"],
                      ua_net.net_edges["from_int"],
                      ua_net.net_edges["to_int"],
                      ua_net.net_edges[["weight"]], 
                      twoway=False)

CPU times: user 1min 53s, sys: 16 s, total: 2min 9s
Wall time: 17.5 s


In [35]:
osm_node_ids = ua_net.net_nodes.loc[ua_net.net_nodes.net_type == 'walk'].index.values
print(len(ua_net.net_nodes))
print(len(osm_node_ids))

59143
55440


In [36]:
osm_nodes = ua_net.net_nodes.reindex(osm_node_ids)  # filters for rows with index in list of values
osm_edges = ua_net.net_edges.loc[ua_net.net_edges.from_int.isin(osm_node_ids) &
                                 ua_net.net_edges.to_int.isin(osm_node_ids)]

In [37]:
%%time
osm_net = pandana.Network(osm_nodes["x"], 
                          osm_nodes["y"],
                          osm_edges["from_int"],
                          osm_edges["to_int"],
                          osm_edges[["weight"]], 
                          twoway=False)

CPU times: user 12.5 s, sys: 892 ms, total: 13.4 s
Wall time: 1.99 s


In [38]:
blocks_subset['node_id'] = osm_net.get_node_ids(blocks_subset['x'], blocks_subset['y'])
net.set(blocks_subset.node_id, variable = blocks_subset.jobs, name='jobs')

In [39]:
jobs_15 = net.aggregate(15, type='sum', decay='linear', name='jobs')

In [40]:
ua_nodes2 = ua_net.net_nodes.copy()
ua_nodes2['jobs_15_act_bart'] = jobs_15

In [41]:
ua_nodes2.head(3)

,id,x,y,net_type,jobs_15_act_bart
id_int,,,,,
1,FTVL_bay_area_rapid_transit_05_bay_area_rapid_...,-122.224274,37.774963,NaN,12455.782702
2,COLS_bay_area_rapid_transit_05_bay_area_rapid_...,-122.197273,37.754006,NaN,9640.885678
3,SANL_bay_area_rapid_transit_05_bay_area_rapid_...,-122.161311,37.722619,NaN,6107.553962


### Merge the accessibilities and compare

We can merge on the original OSM ids, which will match for the OSM nodes.

In [42]:
ua_nodes1_clean = ua_nodes1.loc[ua_nodes1.net_type == 'walk', ['id','x','y','jobs_15_act']]
ua_nodes2_clean = ua_nodes2.loc[ua_nodes2.net_type == 'walk', ['id','jobs_15_act_bart']]

nodes = pd.merge(ua_nodes1_clean, 
                 ua_nodes2_clean, 
                 how = 'inner', 
                 on = 'id')

In [43]:
nodes.head(3)

,id,x,y,jobs_15_act,jobs_15_act_bart
0,30366199,-122.281347,37.828045,2238.892128,2238.892128
1,30366200,-122.280475,37.828248,2569.567284,2569.567284
2,30374146,-122.287497,37.802534,1592.634663,1592.634663


In [44]:
nodes['jobs_15_ratio'] = nodes.jobs_15_act_bart / nodes.jobs_15_act

In [45]:
nodes.describe()

,x,y,jobs_15_act,jobs_15_act_bart,jobs_15_ratio
count,55440.000000,55440.000000,55440.000000,55440.000000,54577.000000
mean,-122.220596,37.784235,2008.193831,2067.731575,1.036367
std,0.055707,0.060465,3741.537578,3851.983741,0.394524
min,-122.342039,37.631875,0.000000,0.000000,1.000000
25%,-122.267031,37.739285,301.864398,302.499099,1.000000
50%,-122.232524,37.787570,898.123474,907.081498,1.000000
75%,-122.174038,37.832181,1898.933564,1973.531040,1.000000
max,-122.107708,37.891111,31560.787784,36738.856966,12.740816


#### The problem is solved: after adding BART to the network, accessibility remains the same or improves at every node.